In [1]:
import pandas as pd

###  Loading data of books we like

In [2]:
#loading our liked books data
my_books = pd.read_csv("liked_books.csv")

#typecasting book_id into string
my_books["book_id"] = my_books["book_id"].astype(str)

#creating set of unique book from our liked_books data
book_set = set(my_books["book_id"])
books = len(book_set)

##  Creating Dictionary from book_mapping dataset to create connection between two dataset

In [4]:
#creating dictionary from csv_book_mapping to connect both dataset based on this

csv_book_mapping = {}

with open("book_id_map.csv",'r') as f:
    while True:
        line = f.readline()
        
        if not line:
            break;
            
        csv_id,book_id = line.strip().split(",")
        
        csv_book_mapping[csv_id] = book_id


##  Getting all the users we have rated books which we like

In [5]:
# finding all users who has read books which we have liked and rated >= 4 starts.
overlap_users = {}

#reading goodreads_interactions data in a streaming fashion
with open("goodreads_interactions.csv") as f:
    i = 0
    while i < 500000:
        i +=1 
        
        #reading single line
        line = f.readline()
        if not line:
            break;
        
        # splitting data by ',' and getting all the columns we care about
        user_id, csv_id, _, rating, _ = line.strip().split(",")
        
        #converting rating into int if not possible ignore that line
        try:
            rating = int(rating)
        except ValueError:
            continue
            
        
        book_id = csv_book_mapping.get(csv_id)
        
        
        if book_id in book_set and rating >= 4:
            if user_id not in overlap_users:
                overlap_users[user_id] = 1
            else:
                overlap_users[user_id] += 1
                

In [6]:
filtered_overlap_users = set(i for i in overlap_users if overlap_users[i] > books/2)

### finding records of book ratings where only those users have rated, who has read all the books which we like


In [33]:
#finding records of book ratings where only those users have rated, who has read all the books which we like
#creating records using overlap_users data

rec_lines = []

with open("goodreads_interactions.csv", 'r') as f:
    i = 0
    while i < 100000:
        i += 1
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",")
        
        if user_id in overlap_users:
            book_id = csv_book_mapping[csv_id]
            if book_id not in book_set:
                rec_lines.append([user_id, book_id, rating])
        

###  Loadind all those records into pandas dataframe

In [34]:
import pandas as pd

recs = pd.DataFrame(rec_lines, columns=["user_id","book_id","rating"])
recs["book_id"] = recs["book_id"].astype(str)

###  Getting top 15 books which is rated by highest number of users who rated books we likes

In [35]:
top_recs = recs["book_id"].value_counts().head(15)
top_recs = top_recs.index.to_list()

###  loading data from books_titles.json files and printing all the top records data from this json file

In [36]:
book_titles = pd.read_json("books_titles.json")
book_titles["book_id"] = book_titles["book_id"].astype(str)

In [46]:
def clickable(val):
    return '<a target="_blank" href="{}">book Link</a>'.format(val)

def show_image(val):
    return '<img src="{}" width=50></img>'.format(val)

In [47]:
#book_titles.head(10).style.format({'url':clickable, 'cover_image' : show_image})

In [48]:
recs = book_titles[book_titles["book_id"].isin(top_recs)] 
len(recs)

7

# top recommended books for us

In [49]:
recs.style.format({'url':clickable, 'cover_image' : show_image})